# 1. Package preparation

In [2]:
#! pip install missingno
#! pip install geopy
# !pip install geopandas
# !pip install geoplot   # need conda install -c conda-forge cartopy 
# !pip install shapely
import numpy as np
import pandas as pd
import geopy
np.set_printoptions(threshold=np.inf)
from scipy import stats
import seaborn as sns
%matplotlib inline
from matplotlib.pylab import plt
%pylab inline
import missingno as msno 
import warnings # ignore the warnings
from geopandas import GeoDataFrame
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopy.geocoders import Nominatim
import geoplot as gplt
from string import ascii_letters
import seaborn as sns
import scipy.stats as ss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [3]:
UShousing2 = pd.read_csv('UShousing2.csv')
UShousing2.shape

(7250, 12)

In [4]:
UShousing3 = UShousing2.copy()
X = UShousing3.drop('rent_amount',axis=1)
y = UShousing2['rent_amount']
X.shape, y.shape

((7250, 11), (7250,))

# 2. Feature Engineering

a. Convert categorical features to numerics
b. Standardize Numerical features

In [5]:
UShousing2 = UShousing2[UShousing2['rent_amount'] > 0]
xreg = pd.get_dummies(data=X[X.columns.difference(["identity"])])

# 3. Linear Regession Model Implement

##### Split Training set & Testing set

In [6]:
X_train, X_test, y_train, y_test = train_test_split(xreg, y, test_size=0.3, random_state= 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5075, 6880), (2175, 6880), (5075,), (2175,))

In [7]:
X_train.shape

(5075, 6880)

In [8]:
numerics = ['int64','float64']

train_num = X_train.select_dtypes(include=numerics)
test_num =  X_test[train_num.columns]


scaler = StandardScaler()
x_train_scaled = pd.DataFrame(scaler.fit_transform(train_num),columns = train_num.columns)
x_test_scaled = pd.DataFrame(scaler.transform(test_num), columns = test_num.columns)

x_train_scaled.shape,x_test_scaled.shape
x_train_scaled.head()

,bathroom_count,bedroom_count,publisher_identity,rent_duration,year
0,-0.557945,-0.620497,-0.148548,-0.001108,-0.148402
1,-0.557945,-0.620497,-0.148548,-0.001108,-0.148402
2,-0.557945,-0.620497,-0.148548,-0.001108,-1.407833
3,0.033568,0.737719,8.617538,-7.160020,1.111029
4,-0.557945,-0.620497,-0.148548,-0.001108,1.111029


In [9]:
X_train.loc[:,'bathroom_count'] = x_train_scaled.loc[:,'bathroom_count'].values
X_train.loc[:,'bedroom_count'] = x_train_scaled.loc[:,'bedroom_count'].values
X_train.loc[:,'rent_duration'] = x_train_scaled.loc[:,'rent_duration'].values
X_test.loc[:,'bathroom_count'] = x_test_scaled.loc[:,'bathroom_count'].values
X_test.loc[:,'bedroom_count'] = x_test_scaled.loc[:,'bedroom_count'].values
X_test.loc[:,'rent_duration'] = x_test_scaled.loc[:,'rent_duration'].values

In [10]:
# simple linear regression 
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
# reg.predict(X_test)
import sklearn.metrics as metrics
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    # mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    # median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    # print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))
regression_results(y_test,reg.predict(X_test))

explained_variance:  -3.357928688125219e+23
r2:  -3.3594543339452725e+23
MAE:  157713731122192.6
MSE:  3.747194543283424e+29
RMSE:  612143328256007.6


In [11]:
## Linear regression with regularization - Lasso
from sklearn.model_selection import cross_val_score, KFold
from sklearn import linear_model
from statistics import mean
alphas = [0.05,0.1,0.15,0.2,0.25]
kf = KFold(3, shuffle=True, random_state=0)
cross_val_list = []

for i in alphas:
    rm2b1 = linear_model.Lasso(alpha=i, fit_intercept = True,max_iter=5000, normalize=True)
    cross_val_list.append(sum(cross_val_score(rm2b1, X_train, y_train, cv=kf)) / 4)

alpha_best_cv = alphas[cross_val_list.index(max(cross_val_list))]
rm2a3 = linear_model.Lasso(alpha=0.21, fit_intercept = True)
lasso_model_2a3 = rm2a3.fit(X_train, y_train)
ytest_pred_2a3 = rm2a3.predict(X_test)
print('MAE: ',  metrics.mean_absolute_error(y_test, ytest_pred_2a3))
print('MSE: ',  metrics.mean_squared_error(y_test, ytest_pred_2a3))

MAE:  585.524355844828
MSE:  1463023.3751264047
